## Bedrock Claude FMEval Example

In this notebook we utilize Claude via Bedrock with the FMEval Library to test the Factual Knowledge Evaluation Algorithm.

Environment:
- conda_python3 kernel
- Studio Notebook instance type: ml.c5.xlarge

### Setup

In [ ]:
import boto3
import json

# Bedrock clients for model inference
bedrock = boto3.client(service_name='bedrock')
bedrock_runtime = boto3.client(service_name='bedrock-runtime')

In [ ]:
import glob

# Check for beta wheel and built-in dataset
if not glob.glob("amazon_fmeval-0.1.0-py3-none-any.whl"):
    print("ERROR - please make sure file exists: new_dist/amazon_fmeval-*-py3-none-any.whl")

if not glob.glob("tiny_dataset.jsonl"):
    print("ERROR - please make sure file exists: tiny_dataset.jsonl")

In [ ]:
#
# Install the amazon_fmeval-*-py3-none-any.whl distribution.
#

#!rm -Rf ~/.cache/pip/*

#!pip3 install amazon_fmeval-0.1.0-py3-none-any.whl --upgrade --upgrade-strategy only-if-needed --force-reinstall
#!pip3 install boto3==1.28.65

### Sample Bedrock Inference

Understand the prompt template we need for evaluation.

In [ ]:
import json

model_id = 'anthropic.claude-v2'
accept = "application/json"
contentType = "application/json"

prompt_data = """Human: Who is Novak Djokovic?

Assistant:
"""

body = json.dumps({"prompt": prompt_data, "max_tokens_to_sample": 500})

response = bedrock_runtime.invoke_model(
    body=body, modelId=model_id, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())
print(response_body.get("completion"))

### FMEval Setup

In [ ]:
from amazon_fmeval.data_loaders.data_config import DataConfig
from amazon_fmeval.model_runners.bedrock_model_runner import BedrockModelRunner
from amazon_fmeval.constants import MIME_TYPE_JSONLINES
from amazon_fmeval.eval_algorithms.factual_knowledge import FactualKnowledge, FactualKnowledgeConfig

#### Data Config Setup

We use the built-in tine_dataset in this case, but you can also optionally bring your own.

In [ ]:
config = DataConfig(
    dataset_name="tiny_dataset",
    dataset_uri="tiny_dataset.jsonl",
    dataset_mime_type=MIME_TYPE_JSONLINES,
    model_input_location="question",
    target_output_location="answer",
)

#### Model Runner Setup

This model runner performs inference across the dataset while running evaluation with the algorithm we're utilizing.

In [ ]:
bedrock_model_runner = BedrockModelRunner(
    model_id=model_id,
    output='completion',
    content_template='{"prompt": $prompt, "max_tokens_to_sample": 500}'
)

### Evaluation Job

In [ ]:
eval_algo = FactualKnowledge(FactualKnowledgeConfig(target_output_delimiter="<OR>"))
eval_output = eval_algo.evaluate(model=bedrock_model_runner, dataset_config=config, 
                                 prompt_template="Human: $feature\n\nAssistant:\n", save=True)

#### Parse Evaluation Reports

In [ ]:
#
# Print the evalaution output.
#

eval_output

In [ ]:
#
# Pretty-print the evalaution output (notice the score).
#

import json
print(json.dumps(eval_output, default=vars, indent=4))

In [ ]:
import pandas as pd

data = []
with open("/tmp/eval_results/factual_knowledge_tiny_dataset.jsonl", "r") as file:
    for line in file:
        data.append(json.loads(line))
df = pd.DataFrame(data)
df['eval_algo'] = df['scores'].apply(lambda x: x[0]['name'])
df['eval_score'] = df['scores'].apply(lambda x: x[0]['value'])
df

In [ ]:
#
# See the raw evaluation results.
#

!cat /tmp/eval_results/factual_knowledge_tiny_dataset.jsonl